**bclassing ndarray**


View casting

In [1]:
import numpy as np

In [2]:
class C(np.ndarray): pass
arr = np.zeros((3,))
c_arr = arr.view(C)
type(c_arr)

__main__.C

Creating new

In [3]:
v = c_arr[1:]
type(v)

__main__.C

In [4]:
v is c_arr

False

Implications for subclassing

In [5]:
class C:
    def __new__(cls, *args):
        print('Cls in __new__:', cls)
        print('Args in __new__:', args)
        # The `object` type __new__ method takes a single argument.
        return object.__new__(cls)
    def __init__(self, *args):
        print('type(self) in __init__:', type(self))
        print('Args in __init__:', args)
c = C('hello')

Cls in __new__: <class '__main__.C'>
Args in __new__: ('hello',)
type(self) in __init__: <class '__main__.C'>
Args in __init__: ('hello',)


In [6]:
class D(C):
    def __new__(cls, *args):
        print('D cls is:', cls)
        print('D args in __new__:', args)
        return C.__new__(C, *args)

    def __init__(self, *args):
        # we never get here
        print('In D __init__')

In [7]:
obj = D('hello')

D cls is: <class '__main__.D'>
D args in __new__: ('hello',)
Cls in __new__: <class '__main__.C'>
Args in __new__: ('hello',)


In [8]:
import numpy as np

class C(np.ndarray):
    def __new__(cls, *args, **kwargs):
        print('In __new__ with class %s' % cls)
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        # in practice you probably will not need or want an __init__
        # method for your subclass
        print('In __init__ with class %s' % self.__class__)

    def __array_finalize__(self, obj):
        print('In array_finalize:')
        print('   self type is %s' % type(self))
        print('   obj type is %s' % type(obj))

In [9]:
c = C((10,))

In __new__ with class <class '__main__.C'>
In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class 'NoneType'>
In __init__ with class <class '__main__.C'>


In [10]:
a = np.arange(10)
cast_a = a.view(C)

In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class 'numpy.ndarray'>


In [11]:
cv = c[:1]

In array_finalize:
   self type is <class '__main__.C'>
   obj type is <class '__main__.C'>


adding an extra attribute to ndarray

In [12]:
import numpy as np

class InfoArray(np.ndarray):

    def __new__(subtype, shape, dtype=float, buffer=None, offset=0,
                strides=None, order=None, info=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = super().__new__(subtype, shape, dtype,
                              buffer, offset, strides, order)
        # set the new 'info' attribute to the value passed
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        self.info = getattr(obj, 'info', None)
        # We do not need to return anything

In [13]:
obj = InfoArray(shape=(3,)) # explicit constructor
type(obj)

__main__.InfoArray

In [14]:
obj.info is None

True

In [15]:
obj = InfoArray(shape=(3,), info='information')
obj.info

'information'

In [16]:
v = obj[1:] # new-from-template - here - slicing
type(v)

__main__.InfoArray

In [17]:
v.info

'information'

In [18]:
arr = np.arange(10)
cast_arr = arr.view(InfoArray) # view casting
type(cast_arr)

__main__.InfoArray

In [19]:
cast_arr.info is None

True

attribute added to existing array

In [1]:
import numpy as np

class RealisticInfoArray(np.ndarray):

    def __new__(cls, input_array, info=None):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        # add the new attribute to the created instance
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # see InfoArray.__array_finalize__ for comments
        if obj is None: return
        self.info = getattr(obj, 'info', None)

In [2]:
arr = np.arange(5)
obj = RealisticInfoArray(arr, info='information')
type(obj)

__main__.RealisticInfoArray

In [3]:
obj.info

'information'

In [5]:
v = obj[1:]
type(v)

__main__.RealisticInfoArray

In [6]:
v.info

'information'

__array_ufunc__ for ufuncs

In [9]:
import numpy as np

class A(np.ndarray):
    def __array_ufunc__(self, ufunc, method, *inputs, out=None, **kwargs):
        args = []
        in_no = []
        for i, input_ in enumerate(inputs):
            if isinstance(input_, A):
                in_no.append(i)
                args.append(input_.view(np.ndarray))
            else:
                args.append(input_)

        outputs = out
        out_no = []
        if outputs:
            out_args = []
            for j, output in enumerate(outputs):
                if isinstance(output, A):
                    out_no.append(j)
                    out_args.append(output.view(np.ndarray))
                else:
                    out_args.append(output)
            kwargs['out'] = tuple(out_args)
        else:
            outputs = (None,) * ufunc.nout

        info = {}
        if in_no:
            info['inputs'] = in_no
        if out_no:
            info['outputs'] = out_no

        results = super().__array_ufunc__(ufunc, method, *args, **kwargs)
        if results is NotImplemented:
            return NotImplemented

        if method == 'at':
            if isinstance(inputs[0], A):
                inputs[0].info = info
            return

        if ufunc.nout == 1:
            results = (results,)

        results = tuple((np.asarray(result).view(A)
                         if output is None else output)
                        for result, output in zip(results, outputs))
        if results and isinstance(results[0], A):
            results[0].info = info

        return results[0] if len(results) == 1 else results

In [10]:
a = np.arange(5.).view(A)
b = np.sin(a)
b.info

{'inputs': [0]}

In [11]:
b = np.sin(np.arange(5.), out=(a,))
b.info

{'outputs': [0]}

In [12]:
a = np.arange(5.).view(A)
b = np.ones(1).view(A)
c = a + b
c.info

{'inputs': [0, 1]}

In [13]:
a += b
a.info

{'inputs': [0, 1], 'outputs': [0]}

__array_wrap__ for ufuncs and other functions

In [14]:
import numpy as np

class MySubClass(np.ndarray):

    def __new__(cls, input_array, info=None):
        obj = np.asarray(input_array).view(cls)
        obj.info = info
        return obj

    def __array_finalize__(self, obj):
        print('In __array_finalize__:')
        print('   self is %s' % repr(self))
        print('   obj is %s' % repr(obj))
        if obj is None: return
        self.info = getattr(obj, 'info', None)

    def __array_wrap__(self, out_arr, context=None):
        print('In __array_wrap__:')
        print('   self is %s' % repr(self))
        print('   arr is %s' % repr(out_arr))
        # then just call the parent
        return super().__array_wrap__(self, out_arr, context)

In [15]:
obj = MySubClass(np.arange(5), info='spam')

In __array_finalize__:
   self is MySubClass([0, 1, 2, 3, 4])
   obj is array([0, 1, 2, 3, 4])


In [16]:
arr2 = np.arange(5)+1
ret = np.add(arr2, obj)

In __array_finalize__:
   self is MySubClass([0, 1, 2, 3, 4])
   obj is MySubClass([0, 1, 2, 3, 4])
In __array_wrap__:
   self is MySubClass([0, 1, 2, 3, 4])
   arr is MySubClass([1, 3, 5, 7, 9])


In [17]:
ret

MySubClass([0, 1, 2, 3, 4])

In [18]:
ret.info

'spam'

In [19]:
class SillySubClass(np.ndarray):

    def __array_wrap__(self, arr, context=None):
        return 'I lost your data'

In [20]:
arr1 = np.arange(5)
obj = arr1.view(SillySubClass)
arr2 = np.arange(5)
ret = np.multiply(obj, arr2)
ret

'I lost your data'

custom __del__ methods and ndarray.base

In [21]:
# A normal ndarray, that owns its own data
arr = np.zeros((4,))
# In this case, base is None
arr.base is None

True

In [22]:
# We take a view
v1 = arr[1:]
# base now points to the array that it derived from
v1.base is arr

True

In [23]:
# Take a view of a view
v2 = v1[1:]
# base points to the original array that it was derived from
v2.base is arr

True

Subclassing and Downstream Compatibility

if you want your sub-class or duck-type to be compatible with numpy’s sum function, the method signature for this object’s sum method should be the following

In [ ]:
def sum(self, axis=None, dtype=None, out=None, keepdims=False):